# Wide & Deep Learning for Recommender System

- Google에서 App Store를 활용해서 발표한 논문([링크](https://arxiv.org/pdf/1606.07792.pdf))


## Google 공식 문서
- Google의 AI Blog([링크](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html))
- Google의 Tensorflow github([링크](https://github.com/tensorflow/tensorflow/blob/v2.4.0/tensorflow/python/keras/premade/wide_deep.py#L34-L219))
- TensorFlow v2.4 API
  - [tf.keras.experimental.WideDeepModel](https://www.tensorflow.org/api_docs/python/tf/keras/experimental/WideDeepModel?hl=en#methods_2)
  - [tf.estimator.DNNLinearCombinedClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNLinearCombinedClassifier)

## 함께볼만한 PyTorch Library
- [pytorch-widedeep](https://github.com/jrzaurin/pytorch-widedeep)

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
path = '../data/kmrd/kmr_dataset/datafile/kmrd-small'

In [3]:
df = pd.read_csv(os.path.join(path,'rates.csv'))
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1234, shuffle=True)



In [4]:
train_df.shape

(112568, 4)

In [5]:
train_df = train_df[:1000]

In [7]:
# Load all related dataframe
movies_df = pd.read_csv(os.path.join(path, 'movies.txt'), sep='\t', encoding='utf-8')
movies_df = movies_df.set_index('movie')

castings_df = pd.read_csv(os.path.join(path, 'castings.csv'), encoding='utf-8')
countries_df = pd.read_csv(os.path.join(path, 'countries.csv'), encoding='utf-8')
genres_df = pd.read_csv(os.path.join(path, 'genres.csv'), encoding='utf-8')

# Get genre information
genres = [(list(set(x['movie'].values))[0], '/'.join(x['genre'].values)) for index, x in genres_df.groupby('movie')]
combined_genres_df = pd.DataFrame(data=genres, columns=['movie', 'genres'])
combined_genres_df = combined_genres_df.set_index('movie')

# Get castings information
castings = [(list(set(x['movie'].values))[0], x['people'].values) for index, x in castings_df.groupby('movie')]
combined_castings_df = pd.DataFrame(data=castings, columns=['movie','people'])
combined_castings_df = combined_castings_df.set_index('movie')

# Get countries for movie information
countries = [(list(set(x['movie'].values))[0], ','.join(x['country'].values)) for index, x in countries_df.groupby('movie')]
combined_countries_df = pd.DataFrame(data=countries, columns=['movie', 'country'])
combined_countries_df = combined_countries_df.set_index('movie')

movies_df = pd.concat([movies_df, combined_genres_df, combined_castings_df, combined_countries_df], axis=1)

print(movies_df.shape)
movies_df.head()

(999, 7)


,title,title_eng,year,grade,genres,people,country
movie,,,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,드라마/멜로/로맨스,"[4374, 178, 3241, 47952, 47953, 19538, 18991, ...","이탈리아,프랑스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,SF/코미디,"[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]",미국
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,SF/코미디,"[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,...",미국
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,서부/SF/판타지/코미디,"[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1...",미국
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,판타지/모험/SF/액션,"[1007, 535, 215, 1236, 35]",미국


In [8]:
movies_df.columns

Index(['title', 'title_eng', 'year', 'grade', 'genres', 'people', 'country'], dtype='object')

In [9]:
dummy_genres_df = movies_df['genres'].str.get_dummies(sep='/')
train_genres_df = train_df['movie'].apply(lambda x: dummy_genres_df.loc[x])
train_genres_df.head()

,SF,가족,공포,느와르,다큐멘터리,드라마,로맨스,멜로,모험,뮤지컬,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
94390,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
dummy_grade_df = pd.get_dummies(movies_df['grade'], prefix='grade')
train_grade_df = train_df['movie'].apply(lambda x: dummy_grade_df.loc[x])
train_grade_df.head()

,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,grade_PG-13,grade_R,grade_전체 관람가,grade_청소년 관람불가
137023,1,0,0,0,0,0,0,0,0
92868,0,0,0,0,1,0,0,0,0
94390,1,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,0,0,1,0
80155,1,0,0,0,0,0,0,0,0


In [19]:
train_df['year'] = train_df.apply(lambda x: movies_df.loc[x['movie']]['year'], axis=1)


In [32]:
train_df = pd.concat([train_df, train_grade_df, train_genres_df], axis=1)
train_df.head()

,user,movie,rate,time,year,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,48423,10764,10,1212241560,1987.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,17307,10170,10,1122185220,1985.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
94390,18180,10048,10,1573403460,2016.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,1498,10001,9,1432684500,2013.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,12541,10022,10,1370458140,1980.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
wide_cols = list(dummy_genres_df.columns) + list(dummy_grade_df.columns)
wide_cols

['SF',
 '가족',
 '공포',
 '느와르',
 '다큐멘터리',
 '드라마',
 '로맨스',
 '멜로',
 '모험',
 '뮤지컬',
 '미스터리',
 '범죄',
 '서부',
 '서사',
 '스릴러',
 '애니메이션',
 '액션',
 '에로',
 '전쟁',
 '코미디',
 '판타지',
 'grade_12세 관람가',
 'grade_15세 관람가',
 'grade_G',
 'grade_NR',
 'grade_PG',
 'grade_PG-13',
 'grade_R',
 'grade_전체 관람가',
 'grade_청소년 관람불가']

In [36]:
print(len(wide_cols))
print(wide_cols)

wide_cols = wide_cols[:3]

30
['SF', '가족', '공포', '느와르', '다큐멘터리', '드라마', '로맨스', '멜로', '모험', '뮤지컬', '미스터리', '범죄', '서부', '서사', '스릴러', '애니메이션', '액션', '에로', '전쟁', '코미디', '판타지', 'grade_12세 관람가', 'grade_15세 관람가', 'grade_G', 'grade_NR', 'grade_PG', 'grade_PG-13', 'grade_R', 'grade_전체 관람가', 'grade_청소년 관람불가']


In [37]:
wide_cols

['SF', '가족', '공포']

In [38]:
import itertools
from itertools import product  
unique_combinations = list(list(zip(wide_cols, element)) 
                           for element in product(wide_cols, repeat = len(wide_cols))) 

print(unique_combinations)
cross_cols = [item for sublist in unique_combinations for item in sublist]
cross_cols = [x for x in cross_cols if x[0] != x[1]]
cross_cols = list(set(cross_cols))
print(cross_cols)

[[('SF', 'SF'), ('가족', 'SF'), ('공포', 'SF')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '가족')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '공포')], [('SF', 'SF'), ('가족', '가족'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '가족'), ('공포', '가족')], [('SF', 'SF'), ('가족', '가족'), ('공포', '공포')], [('SF', 'SF'), ('가족', '공포'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '공포'), ('공포', '가족')], [('SF', 'SF'), ('가족', '공포'), ('공포', '공포')], [('SF', '가족'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '가족'), ('가족', 'SF'), ('공포', '가족')], [('SF', '가족'), ('가족', 'SF'), ('공포', '공포')], [('SF', '가족'), ('가족', '가족'), ('공포', 'SF')], [('SF', '가족'), ('가족', '가족'), ('공포', '가족')], [('SF', '가족'), ('가족', '가족'), ('공포', '공포')], [('SF', '가족'), ('가족', '공포'), ('공포', 'SF')], [('SF', '가족'), ('가족', '공포'), ('공포', '가족')], [('SF', '가족'), ('가족', '공포'), ('공포', '공포')], [('SF', '공포'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '공포'), ('가족', 'SF'), ('공포', '가족')], [('SF', '공포'), ('가족', 'SF'), ('공포', '공포')], [('SF', '공포'), ('가족', '가족'), ('공포', 'SF')], [('SF', '공포'), ('가족', '가족'), ('

In [39]:
# embed_cols = [('genre', 16),('grade', 16)]
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['year']

print(embed_cols)
print(continuous_cols)

[('SF', 16), ('가족', 16), ('공포', 16)]
['year']


In [40]:
target = train_df['rate'].apply(lambda x: 1 if x > 9 else 0).values

## Wide & Deep

In [67]:
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep import Trainer

### Wide Component

In [52]:
wide_cols

['SF', '가족', '공포']

In [53]:
cross_cols

[('공포', 'SF'),
 ('가족', '공포'),
 ('SF', '가족'),
 ('공포', '가족'),
 ('SF', '공포'),
 ('가족', 'SF')]

In [49]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = preprocess_wide.fit_transform(train_df)
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [50]:
X_wide

array([[ 1,  3,  5, ..., 18, 21, 25],
       [ 1,  3,  5, ..., 18, 21, 25],
       [ 1,  3,  5, ..., 18, 21, 25],
       ...,
       [ 1,  3,  5, ..., 18, 21, 25],
       [ 1,  3,  5, ..., 18, 21, 25],
       [ 1,  3,  5, ..., 18, 21, 25]])

In [54]:
X_wide.shape

(1000, 9)

In [51]:
wide

Wide(
  (wide_linear): Embedding(29, 1, padding_idx=0)
)

### Deep Component

In [60]:
embed_cols

[('SF', 16), ('가족', 16), ('공포', 16)]

In [61]:
continuous_cols

['year']

In [58]:
preprocess_deep = TabPreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols)
X_deep = preprocess_deep.fit_transform(train_df)
deepdense = TabMlp(
    mlp_hidden_dims=[64, 32],
    column_idx=preprocess_deep.column_idx,
    embed_input=preprocess_deep.embeddings_input,
    continuous_cols=continuous_cols,
)

/Users/ohyeji/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
X_deep

array([[ 1.        ,  1.        ,  1.        , -0.8555441 ],
       [ 1.        ,  1.        ,  1.        , -0.98182694],
       [ 1.        ,  1.        ,  1.        ,  0.97555713],
       ...,
       [ 1.        ,  1.        ,  1.        ,  0.97555713],
       [ 1.        ,  1.        ,  1.        ,  0.91241571],
       [ 1.        ,  1.        ,  1.        ,  0.40728434]])

In [63]:
X_deep.shape

(1000, 4)

In [59]:
deepdense

/Users/ohyeji/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TabMlp(
  (embed_layers): ModuleDict(
    (emb_layer_SF): Embedding(3, 16, padding_idx=0)
    (emb_layer_가족): Embedding(3, 16, padding_idx=0)
    (emb_layer_공포): Embedding(3, 16, padding_idx=0)
  )
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (tab_mlp): MLP(
    (mlp): Sequential(
      (dense_layer_0): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=49, out_features=64, bias=True)
        (2): ReLU(inplace=True)
      )
      (dense_layer_1): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=64, out_features=32, bias=True)
        (2): ReLU(inplace=True)
      )
    )
  )
)

### Build and Train

In [70]:
# build, compile and fit
model = WideDeep(wide=wide, deeptabular=deepdense)
trainer = Trainer(model, objective="binary", metrics=[Accuracy])
trainer.fit(
    X_wide=X_wide,
    X_tab=X_deep,
    target=target,
    n_epochs=5,
    batch_size=256,
    val_split=0.1,
)

valid: 100%|██████████| 1/1 [00:00<00:00, 16.22it/s, loss=nan, metrics={'acc': 0.0}]


In [71]:
X_deep.shape

/Users/ohyeji/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1000, 4)

In [72]:
X_wide.shape

(1000, 9)